In [53]:
import os
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv
from icecream import ic
import json
from datetime import datetime
from copy import deepcopy
load_dotenv()

True

## load variables - terraces
- data folder
- cleaned_df - for only teracces
- json template
- 

In [54]:
data_folder = os.path.join(os.getcwd(),'data')
cleaned_csv = os.path.join(data_folder, 'final_range_data.csv')
dff = pd.read_csv(cleaned_csv)

#lambda function to filter for terraces
#for each row, is it a list/str, then is 'terraces' in it, if so, return the df.
cleaned_df = dff[
    dff["land_use_selection-intervention"]
    .apply(lambda x: isinstance(x, (list, str)) and "terracing" in x)
]

#terracing datasets, not there, so not needed
#terraces_df = pd.read_csv(os.path.join(data_folder, 'brics_csvs', 'outputs','cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_terraces.csv'))

TEMPLATE_FILE = os.path.join(data_folder,"halfmoon_json_template.json")
with open(TEMPLATE_FILE, "r") as f:
    template = json.load(f)



In [55]:
x = dff["land_use_selection-intervention"]
count = 0
for idx, i in enumerate(list(x)):
    if 'terracing' in i:
        count += 1
        ic(i, count)

ic('terracing count',len(cleaned_df))

ic| i: 'terracing', count: 1
ic| i: 'terracing', count: 2
ic| i: 'terracing', count: 3
ic| 'terracing count', len(cleaned_df): 3


('terracing count', 3)

## enumerators

In [56]:
#sorted(list(dff['basic_info-name'].unique()))
##empty name and key dictionary
names_dict = {}
for idx, row in dff.iterrows():
    #ic({row["KEY"], row["basic_info-name"]})
    names_dict[row["KEY"]] = row["basic_info-name"]
#ic(names_dict)
name_uname_dict = {
    "Zakariye haji ali": "abdullahi.hassan@mardo.org", 
    "Issack Idow Ibrahim": "abdullahi.hassan@mardo.org",
    "Abdullahi Hassan Adan": "abdullahi.hassan@mardo.org",
    "Abdikheir Mohamed": "mukhtar.yusuf23",
    "Abdikheir Mohamed Ali": "mukhtar.yusuf23",
    "Abdikheir Mohamed Dahir": "mukhtar.yusuf23",
    "Ali": "mukhtar.yusuf23",
    "Ali Adan": "mukhtar.yusuf23",
    "Ali Aden": "mukhtar.yusuf23",
    "Fathi": "mukhtar.yusuf23", 
    "Fathi Abdirashid": "mukhtar.yusuf23",
    "Ahmed Omar Hassan": "AhmedOmar",
    "Maria Abdirahman Ali": "AhmedOmar",
    "ABSHIR ABDULLAH ALI": "Ahil",
    "Ahmed Ibrahim": "JubaFoundation",
    "Ahmed Ibrahim Mohamed": "JubaFoundation",
    "Abdiaziz Adan Hassa": "guudow",
    "Abdiaziz Adan Hassan": "guudow",
    "Abdi Hassan Adan": "AbdiHassan",
    "Abdullahi": "sharif1234",
    "Abdirashid Sheikh Mohamed": "NRMO",
    "Abdullahi Sharif": "sharif1234",
    "Abdullahi Sharif LLG": "sharif1234",
    "Abdullahi Sharif Noor LLG": "sharif1234",
    "Abdijalil": "Csxaashi",
    "Abdijalil said": "Csxaashi",
    "Abdikarim Mohamed Ismail": "samandoulgou",
    "Abdikarin Mohamed Ismail": "samandoulgou",
    "Hassan Omar Khadhib": "Khadhib",
    "Hassan Omar Khadhibr": "Khadhib",
    "Jamal Ali": "JamalAli",
    "Jamal Ali Tagal": "JamalAli",
    "Mohamud Abdullahi Ali": "Mohamud",
    "Moulid Abdullahi Abdi": "Moulidwadani",
    "Muktar Mohamednoor Rage": "Mrage1997",
    "Osman Ebey Omar": "iibey",
    "Yusuf Mahi munye": "Mcnbdn617660",
    "abdullahi Sharif LLG": "sharif1234",
    "osman Ebey Omar": "iibey",
}

## match and replace names with usernames
for k, v in names_dict.items():
    for key, value in name_uname_dict.items():
        if v == key:
            names_dict[k] = value
#ic(names_dict)

##enumerator dictionary
enumerator = {
        "id": 3726,
        "first_name": "First",
        "last_name": "Last",
        "gender": "Male",
        "age_category": "18 - 35",
        "phone_number": "0e15704046",
        "email": "test_mail@gmail.com",
        "country": "test",
        "organization": "Test",
        "username": "test",
        "type": "ENUMERATOR"
    }

enums = []
#create enumerator jsons
for key, value in names_dict.items():
    e = deepcopy(enumerator)
    e['username'] = value
    e['key'] = key
    enums.append(e)

#ic(enums)

## farming entity

In [57]:
farming_entity_list = []
for idx, i in enumerate(cleaned_df.to_dict(orient='records'), start=1):
    farming_entity = {}
    #ic(i['basic_info-organisation_name'])
    farming_entity['id'] = idx
    farming_entity['first_name'] = i['basic_info-organisation_name']
    farming_entity['middle_name'] = i['basic_info-organisation_name']
    farming_entity['last_name'] = i['basic_info-organisation_name']
    farming_entity['organization'] = 'INSTITUTION'
    farming_entity['key'] = i['KEY']
    
    farming_entity_list.append(
        {
            "farmingEntity": farming_entity, 
            'key': i['KEY']
        }
    )
    #ic(farming_entity_list)
    

# plot-terracing details
- read df to dictionary, for matching keys, extract df values
### plot-points
- parse the geoshape column from cleaned df
- split and extract the values from each i
- append to json keys

## crops
not present


In [58]:
import uuid
import copy
import re

plot_list = []
point_id = 1

records = cleaned_df.to_dict(orient='records')

count = 0
#iterate over every i to capture plot details
for idx, i in enumerate(records):
    plot_id = idx + 1
    count += 1

    # ---- BUILD PLOT DETAILS ----
    template_cp = copy.deepcopy(template[0])
    plot_details = template_cp['plot']

    plot_details['id'] = plot_id
    plot_details['key'] = i['KEY']
    plot_details['name'] = uuid.uuid4().hex
    plot_details['calculated_size'] = i['area_ha']
    plot_details['photo_url'] = i['plot_details-photo']
    plot_details['farmingEntityId'] = farming_entity['id']
    plot_details['livestock_allowed'] = False if i['land_use_selection-land_use'] == 'agriculture' else True
    plot_details['conservation_area'] = False


    #holds every row's crops
    all_crops = []
    """----------------------------------CROP STATUS ----------------------------------"""
    #not in child table, so skipped
        
    """--------------------------------GEOSHAPE PARSING ----------------------------------"""
    geoshape_str = i['plot_details-polygon']
    
    def parse_geoshape(geoshape_str):
        #parse a geoshape string into a list of point dictionaries
        points = []
        parts = geoshape_str.split(';')

        for part in parts:
            part = part.strip()
            if not part:
                continue

            lat, lon, alt, acc = map(float, part.split(' '))

            points.append({
                "longitude": lon,
                "latitude": lat,
                "altitude": alt,
                "accuracy": acc
            })

        return points

    parsed_points = parse_geoshape(geoshape_str)

    points_list = []
    for p in parsed_points:
        points_list.append({
            "id": point_id,
            "plotId": plot_id,
            "longitude": p["longitude"],
            "latitude": p["latitude"],
            "altitude": p["altitude"],
            "accuracy": p["accuracy"]
        })
        point_id += 1


    """--------------------------------ADMINISTRATIVE DETAILS ----------------------------------"""
    administrative = plot_details['subCounty']
    administrative['subcounty_name'] = i['geography-district_name']
    administrative['county']['county_name'] = i['geography-region_name']


    #update plot_details with number of points and administrative info
    plot_details['points'] = points_list
    plot_details['subCounty'] = administrative
    
    # del plot_details['points']
    # del plot_details['subCounty']
    general_plot = {
        "plot":plot_details, 
        'key':i['KEY']
    }
    #ic(i['KEY'])
    plot_list.append(
        general_plot
    )


## project

In [59]:
projects_list = []

names =[]
for row in cleaned_df.to_dict(orient='records'):
    name = row['land_use_selection-project_name']
    
    if name == 'terra':
        name = name.upper()
        id = 131
    elif name == 'brcis3':
        name = 'BRCiS III'
        id = 93
    
    names.extend([name])
    
    projects_list.append({
        "project": {
            "id": id,
            "project_name": name
        },
        "key": row['KEY']
    })

ic(set(names))


ic| set(names): {'BRCiS III'}


{'BRCiS III'}

## Terracing
- no info whatsoever in child table.
- only take from parent

In [60]:
terraces_list = []

#dictionary to map odk values to RA app schema values
mapping_dict = {
    "STICKS_BRANCH":"STICKS_PROTECTION",
    "DEBRIS_REMOVAL":"DEBRIS_REMOVAL",
    "INVASIVE_SPECIES":"INVASIVE_CONTROL",
    "WEEDING":"WEEDING",
    "WATERING":"WATERING",
    "PLANTING_SEEDLING":"PLANTING_ANNUAL_PERENNIAL_CROPS"
}

for idx, c in enumerate(records):

    """-----------------------------------BUNDS CHILD DETAILS-------------------------------------"""
    #not present
    """-----------------------------------Terracing DETAILS-------------------------------------"""

    terraces_details = {
        "id": idx + 1,
        "key":c["KEY"],
        "erosion_control_type": "Terraces",
        "established_date": datetime.strptime(c['terracing-terrace_details-date'], "%d/%m/%Y").strftime("%Y-%m"),
        "material_used": [
            "OTHER"
        ],
        "other_material_used": "MIGRATED_NOT_KNOWN",
        "who_established": ["MALE", "FEMALE"] if c["terracing-terrace_details-manages"] == 'both' else "MALE" if c["terracing-terrace_details-manages"]=="male" else "",
        "other_who_established": "YOUTH" if c["terracing-terrace_details-youth_manages"] == 'yes' else "",
        "total_interventions": c["terracing-terrace_details-total"],
        "length": c["terracing-terrace_details-length"],
        "width": c["terracing-terrace_details-width"],
        "depth": c["terracing-terrace_details-depth"],
        "vertical_spacing": 0,
        "horizontal_spacing": 0,
        "labor_payment": c["terracing-terrace_details-paid_unpaid_labor"],        
        "has_grass": False,
        "total_number_grasses_planted": 0,
        "kg_grass_seeds_planted": 0,
        "has_trees": False,
        "total_number_different_trees_planted": 0,
        "total_number_trees_planted": 0,
        "total_number_trees_survived": 0,
        "management_practices":  [],
        "other_managements": " ",
        "usages":["MIGRATED_NOT_KNOWN"],
        "other_usages": "",
        "rangelandEntryId": 2,
        "currentStatus": [],
        "grassEstablishment": [],
        "treeEstablishment":[]  
    }

    terraces_list.append(terraces_details)
    


bunds_econtrol = [{"econtrol": [i], "key" :i["key"]} for i in terraces_list]

In [61]:
with open(os.path.join(data_folder, 'to_delete.json'), 'w') as f:
    json.dump(bunds_econtrol, f, indent=4)

## merge components into one json

In [62]:
# component lists
lists = [farming_entity_list, bunds_econtrol,plot_list, projects_list]

# The output dictionary
merged = {}

for lst in lists:
    ic(len(lst))
    for item in lst:
        k = item['key']

        #check if already updated
        if k not in merged:
            merged[k] = {}
            
        #merge the item into the existing dictionary
        merged[k].update(item)

final = list(merged.values())
for i in final:
    #match enumerators
    for l in enums:
        if i['key'] == l['key']:
            i['enumerator'] = l


    i["boma"] = []
    i["cgrazing"] = []
    i["microcatchment"]= []
    i["seedbank"] = []
    i["seeding"] = []
    i["iremoval"] = []
    i["waterpoint"] = []
    i["completed"] = True
    i["rangeModuleId"] = 1
    i["currentStep"]= "DONE"
    i["moduleId"] = 6
    i["is_revisit"] = False
    #get the date_collected
    for j in records:
        if i["key"] == j["KEY"]:
            i["date_collected"] = datetime.strptime(j['basic_info-date'], "%d/%m/%Y").strftime("%Y-%m-%d")
    #match project id
    for x in projects_list:
        if i['key'] == x['key']:
            i['projectId'] = x['project']['id']
    #match plot id
    for k in plot_list:
        if i['key'] == k['key']:
            i['plotId'] = k['plot']['id']
    #match farming_entity_id
    for h in farming_entity_list:
        if i['key'] == h['key']:
            i['farmingEntityId'] = h['farmingEntity']['id']
    #match enumerator id
    for e in enums:
        if i['key'] == e['key']:
            i['enumeratorId'] = e['id']


ic(len(final))


ic| len(lst): 3
ic| len(lst): 3
ic| len(lst): 3
ic| len(lst): 3
ic| len(final): 3


3

In [63]:
with open(os.path.join(data_folder, 'to_delete.json'), 'w') as f:
    json.dump(final, f, indent=4)

## clean up keys

In [64]:
#clean up keys
def remove_keys(obj, key="key"):
    if isinstance(obj, dict):
        obj.pop(key, None)
        for v in obj.values():
            remove_keys(v, key)
    elif isinstance(obj, list):
        for v in obj:
            remove_keys(v, key)
    return obj

finaal = [x for x in final if x['key'] == 'uuid:b4a2d545-66fe-4d23-825b-41025149b460']
finaal = [remove_keys(i) for i in final]

#add id
for idx, c in enumerate(finaal):
    c['id'] = idx + 1



In [65]:
with open(os.path.join(data_folder, 'to_delete.json'), 'w') as f:
    json.dump(finaal, f, indent=4)

In [66]:
for i in enumerate(finaal):
    if len(i[1].keys()) != 23:
        ic("ERROR")
    else:
        ic("✅")


ic| '✅'
ic| '✅'
ic| '✅'
